In [97]:
import torch
from torch import nn
from torch.nn import functional as F
import pytorch_lightning as pl
import math

In [98]:
params = {}

In [225]:
hparams = {'accelerator': 'gpu',
           'devices': [0],
           'dtype': torch.float64,
           'precision': 64,
           'loss_terms': [F.mse_loss],
           'loss_coeffs': [1],
           'optimizer': torch.optim.Adam, 
           'learning_rate': 1e-4,
           'batch_size': 100,
           'max_epochs': 5000,
           'early_stopping_patience': 1000000,
           'DeepONet': False,
           'VarMiON': True,
           'NGO': False,
           'Q': 12,
           'Q_L': 268,
           'h': 64, 
           'Petrov-Galerkin': False,
           'bias_NLBranch': False,
           'bias_LBranch': False,
           'NLB_outputactivation': nn.Tanhshrink(),
           'Cholesky': False,
           'scale_invariance': False,
           'norm_basis': True,
           'bound_mus': False,
           '1/theta': False,
           'symgroupavg': False,
           'NOMAD': False}

params['hparams'] = {}
params['hparams'] = hparams

In [226]:
class ReshapeLayer(nn.Module):
    def __init__(self, output_shape):
        super().__init__()
        self.output_shape = output_shape
    
    def forward(self, x):
        y = x.reshape(self.output_shape)
        return y

In [227]:
Q = 12
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [829]:
class PConv(nn.Module):
    def __init__(self, hidden_channels, kernel_size, stride, bias):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=hidden_channels, kernel_size=kernel_size, stride=stride, bias=bias))
        self.layers.append(nn.BatchNorm2d(num_features=hidden_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(in_channels=hidden_channels, out_channels=1, kernel_size=kernel_size, stride=stride, bias=bias))
    
    def forward(self, x):
        P = x
        for layer in self.layers:
            P = layer(P)
        P = P/torch.norm(P)
        y = x + torch.matmul(P,x)
        return y

In [830]:
model = PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False)

In [831]:
sum(p.numel() for p in model.parameters())

2176

In [832]:
model

PConv(
  (layers): ModuleList(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

In [833]:
y = model(x)
y.shape

RuntimeError: Given groups=1, weight of size [64, 1, 4, 4], expected input[1, 100, 64, 64] to have 1 channels, but got 100 channels instead

In [842]:
class NLBranch_NGO(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.layers = nn.ModuleList()
        self.layers.append(ReshapeLayer((hparams['batch_size'],1,hparams['h'],hparams['h'])))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(PConv(hidden_channels=64, kernel_size=4, stride=1, bias=False))
        self.layers.append(ReshapeLayer((hparams['batch_size'],64,64)))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        y = x
        return y

In [843]:
model2 = NLBranch_NGO(params)

In [844]:
sum(p.numel() for p in model2.parameters())

21760

In [837]:
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [838]:
model2.forward(x)

tensor([[[ 1.2660e-01,  9.5539e-03,  9.8869e-03,  ...,  1.5795e-01,
           9.2218e-02,  2.5349e-03],
         [ 2.4445e-02,  1.5745e-04,  3.8665e-02,  ...,  1.2233e-03,
           3.9848e-02,  1.7122e-01],
         [ 7.0024e-02,  6.7726e-02,  2.3216e-01,  ...,  6.3851e-02,
           3.4230e-04,  1.7863e-06],
         ...,
         [ 3.6762e-03,  6.2164e-02,  1.4455e-01,  ...,  1.8122e-01,
           9.0918e-02,  2.0407e-01],
         [ 1.1252e-01,  7.3541e-03,  5.1887e-02,  ...,  5.3736e-02,
           1.8951e-01,  8.5818e-02],
         [ 1.2263e-01,  1.2239e-01,  6.7225e-03,  ...,  2.2892e-04,
           2.2715e-01,  8.5528e-03]],

        [[ 8.5735e-03,  2.4894e-04,  1.6277e-01,  ...,  1.9619e-01,
           1.4172e-02,  6.7354e-02],
         [ 2.0241e-02,  3.6662e-02,  5.2415e-03,  ...,  1.3689e-01,
           6.0724e-03,  1.8813e-01],
         [ 6.1002e-07,  1.1386e-01,  3.5557e-03,  ...,  1.1700e-02,
           1.7000e-01,  2.2695e-01],
         ...,
         [ 1.4429e-04,  1